# <center>CSE 258 Fall 2017 - Homework 1<center>
- - -

## <center>Xiangyuan Ren<center>
### <center> Department of Electrical and Computer Engineering <center>
### <center>Email: xir010@eng.ucsd.edu <center>

## Regression (*Week 1*)
### Answers Briefly (*Details shown in following pages*)
* 
#### Q1:
* Hefeweizen- 618- 3.63430420712   
* English Strong Ale-164 - 3.74390243902
* Foreign / Export Stout-55 - 3.21818181818
* German Pilsener-586- 3.6638225256
#### Q2:
* $\theta0$ = 3.91520474,$\theta1$= 0.08564622
#### Q3:
* MSE on training set is 0.55810729
* MSE on test set is 0.46841005
#### Q4:
* MSE on training set is 0.36784045
* MSE on test set is 0.43366651

In [1]:
#Reading data 
import numpy
import random
def ParseDataFromFile(f):
    for l in open(f):
        yield eval(l)
data=list(ParseDataFromFile("beer_50000.json"))
data[0]

{'beer/ABV': 5.0,
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'beer/name': 'Sausa Weizen',
 'beer/style': 'Hefeweizen',
 'review/appearance': 2.5,
 'review/aroma': 2.0,
 'review/overall': 1.5,
 'review/palate': 1.5,
 'review/taste': 1.5,
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
 'review/timeStruct': {'hour': 20,
  'isdst': 0,
  'mday': 16,
  'min': 57,
  'mon': 2,
  'sec': 3,
  'wday': 0,
  'yday': 47,
  'year': 2009},
 'review/timeUnix': 1234817823,
 'user/profileName': 'stcules'}

---
## Qestion 1: 
###            How many reviews are there for each style of beer in the dataset (‘beer/style’)? What is the average value of ‘review/taste’ for reviews from each style?
*  **style** (*list*) to store the Name of each style( it is ordered)
*  **styleNum** (*dictionary*) to store the Number of review for each style
*  **styleAverage** (*dictionary*) to store the average value of 'review/taste' for these reviews

In [2]:
style=[]
for i in range(50000):
    if data[i]['beer/style'] not in style:
        style.append(data[i]['beer/style'])
styleNum={}
styleValueSum={}
styleAverage={}
for i in range(50000):
    if data[i]['beer/style'] not in styleNum:
        styleNum[data[i]['beer/style']]=1
        styleValueSum[data[i]['beer/style']]=1
    else:
        styleNum[data[i]['beer/style']]+=1
        styleValueSum[data[i]['beer/style']]+=data[i]['review/taste']
for each in styleValueSum:
    styleAverage[each]=styleValueSum[each]/styleNum[each]
    
print style[:5]
print styleNum[style[0]],styleNum[style[1]],styleNum[style[2]],styleNum[style[3]],styleNum[style[4]]
print styleAverage[style[0]],styleAverage[style[1]],styleAverage[style[2]],styleAverage[style[3]],styleAverage[style[4]]

['Hefeweizen', 'English Strong Ale', 'Foreign / Export Stout', 'German Pilsener', 'American Double / Imperial IPA']
618 164 55 586 3886
3.63430420712 3.74390243902 3.21818181818 3.6638225256 4.03242408646


### So, here is the result for some styles:
| Hefeweizen | English Strong Ale | Foreign / Export Stout | German Pilsener|American Double / Imperial IPA|
|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|
| 617 | 163 | 54 | 585 | 3885 |
| 3.63430420712 |3.74390243902 | 3.21818181818 | 3.6638225256 | 4.03242408646 |

---
## Qestion 2: 
### Train a simple predictor with a single binary feature indicating whether a beer is an ‘American IPA’
### Report the values of $\theta0$ and $\theta1$. Briefly describe your interpretation of these values, i.e., what do $\theta1$ and $\theta1$ represent

In [3]:
def feature(datum):
    if datum['beer/style']=="American IPA":
        feat=[1,1]
    else:
        feat=[1,0]
    return feat
X= [feature(d) for d in data]
y=[d['review/taste']for d in data]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)
theta

array([ 3.91520474,  0.08564622])

### $\theta0$ = 3.91520474,$\theta1$= 0.08564622
* $\theta0$: it represent the **average value** of the 'review/taste' (Almost every beer has 3.915 average 'review/taste')
* $\theta1$: it represent the weight of 'beer/style'(American IPA has 0.0856 higher 'review/taste')

---
## Qestion 3: 
### Split the data into two equal fractions – the first half for training, the second half for testing
### What is the model’s MSE on the training and on the test set

In [4]:
X1 = numpy.matrix([feature(d) for d in data[:25000]])
X2 = numpy.matrix([feature(d) for d in data[25000:]])
y1 = numpy.matrix([d['review/taste']for d in data[:25000]])
y2 = numpy.matrix([d['review/taste']for d in data[25000:]])

theta1,residuals,rank,s = numpy.linalg.lstsq(X1, y1.T)

MSEtrain=(y1.T-X1*theta1).T*(y1.T-X1*theta1)/25000
MSEtest=(y2.T-X2*theta1).T*(y2.T-X2*theta1)/25000
print MSEtrain,MSEtest

[[ 0.55810729]] [[ 0.46841005]]


### Using numpy.linalg.lstsg and spliting dataset:
* MSE on training set is 0.55810729
* MSE on test set is 0.46841005

---
## Qestion 4: 
### Extend the model above so that it incorporates binary features for every style of beer with ≥ 50 reviews
### Report the values of $\theta$ that you obtain, and the model’s MSE on the training and on the test set

In [5]:
for key in style[:]:
    if styleNum[key]<50:# remove style has reviews<=50
        style.remove(key)
        styleNum.pop(key)
print len(styleNum)#how many styles left: 74

74


In [6]:
def feature(datum):
    feat=[0]*(1+74)
    feat[0]=1
    if datum['beer/style'] in style:
        feat[style.index(datum['beer/style'])]=1
    return feat
X= [feature(d) for d in data]
y=[d['review/taste']for d in data]
print X[1],'\n',y[1]#show an sample of feature X and label y

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
3.0


In [7]:
X1 = numpy.matrix(X[:25000])
X2 = numpy.matrix(X[25000:])
y1 = numpy.matrix(y[:25000])
y2 = numpy.matrix(y[25000:])
theta1,residuals,rank,s = numpy.linalg.lstsq(X1, y1.T)
MSEtrain=(y1.T-X1*theta1).T*(y1.T-X1*theta1)/25000
MSEtest=(y2.T-X2*theta1).T*(y2.T-X2*theta1)/25000
print MSEtrain,MSEtest

[[ 0.36784045]] [[ 0.43366651]]


### Containg style with review>=50 as feature:
* MSE on training set is 0.36784045
* MSE on test set is 0.43366651